In [2]:
# Neural networks offer great flexibility and power, but this comes at the cost of many tunable hyperparameters.
# Choosing the right values for these hyperparameters can make a significant difference in model performance.
# This script explores techniques for hyperparameter tuning using the California Housing dataset.
# We'll prepare the data and build a framework for training, validating, and evaluating neural networks,
# with the goal of finding an optimal configuration for this specific regression task.

# Let's start by importing the California Housing dataset from Scikit-Learn
from sklearn.datasets import fetch_california_housing  # Load the California housing dataset
from sklearn.model_selection import train_test_split    # Tool to split data into train/validation/test sets
from sklearn.preprocessing import StandardScaler        # Tool to scale features to standard normal distribution

# Load the dataset (features and target)
housing = fetch_california_housing()

# Split data into training and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

# Further split training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

# Standardize the features: zero mean and unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)      # Fit on training data and transform it
X_valid = scaler.transform(X_valid)          # Use the same transformation on validation data
X_test = scaler.transform(X_test)            # Use the same transformation on test data

In [12]:
# Import Keras from TensorFlow to build and train deep learning models
from tensorflow import keras

# Define a function that builds a customizable neural network model
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=(8,), **kwargs):
    # Create a Sequential model (a linear stack of layers)
    model = keras.models.Sequential()
    
    # Add an input layer with the specified shape
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    
    # Add the specified number of hidden layers, each with ReLU activation
    for layer in range(n_hidden): # for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    
    # Add an output layer with a single neuron (for regression)
    model.add(keras.layers.Dense(1))
    
    # Use stochastic gradient descent (SGD) optimizer with the specified learning rate
    # clipnorm is used to avoid the Keras Regressor from diverging during training
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate, clipnorm=1.0)
    
    # Compile the model with mean squared error loss (appropriate for regression tasks)
    # Add metrics=["mae"] to pass on a metric otherwise KerasRegressor throws the following error:
    # ValueError: Could not interpret metric identifier: loss
    model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["mae"])
    
    # Return the compiled model
    return model  

In [13]:
from scikeras.wrappers import KerasRegressor

# Wrap the Keras model into a Scikit-Learn compatible regressor
# This allows you to use Scikit-Learn tools like cross-validation and grid search
keras_reg = KerasRegressor(
    model=build_model,
    model__n_hidden=2,
    model__n_neurons=30,
    model__learning_rate=0.01,
    model__input_shape=(X_train.shape[1],)  # shape should match input data
    #metrics=None  # Prevents Scikeras from misinterpreting the loss
)

# Train the model on the training data
# - Run for up to 100 epochs
# - Use the validation set to monitor overfitting
# - Apply early stopping to halt training if validation loss doesn't improve for 10 consecutive epochs
keras_reg.fit(
    X_train, y_train,
    epochs=100,
    validation_data=(X_valid, y_valid),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
)

Epoch 1/100
363/363 [==============================] - 2s 3ms/step - loss: 1.0797 - mae: 0.7274 - val_loss: 0.5607 - val_mae: 0.5483
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5191 - mae: 0.5225 - val_loss: 0.4875 - val_mae: 0.5149
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4627 - mae: 0.4926 - val_loss: 0.4556 - val_mae: 0.4766
Epoch 4/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4399 - mae: 0.4767 - val_loss: 0.4392 - val_mae: 0.4798
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4238 - mae: 0.4684 - val_loss: 0.4262 - val_mae: 0.4686
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4132 - mae: 0.4605 - val_loss: 0.4204 - val_mae: 0.4705
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4058 - mae: 0.4562 - val_loss: 0.4139 - val_mae: 0.4590
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - 

KerasRegressor(
	model=<function build_model at 0x00000246817B9D00>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	model__n_hidden=2
	model__n_neurons=30
	model__learning_rate=0.01
	model__input_shape=(8,)
)

In [14]:
# Evaluate the model's performance on the test set
mse_test = keras_reg.score(X_test, y_test)
print(f"Mean Squared Error on test set: {mse_test:.2}")

# Select the first 3 samples from the test set to simulate new/unseen input data
X_new = X_test[:3]

# Use the trained model to predict the target values for these new samples
y_pred = keras_reg.predict(X_new)
print(f"Predictions for new samples: {y_pred}")

162/162 [==============================] - 0s 2ms/step
Mean Squared Error on test set: 0.78
1/1 [==============================] - 0s 32ms/step
Predictions for new samples: [2.4165788 1.3119974 0.9692234]


In [15]:
import numpy as np
# For sampling learning rates from a logarithmic distribution
from scipy.stats import reciprocal
# Tool to search over hyperparameter space using randomized sampling
from sklearn.model_selection import RandomizedSearchCV

# Define distributions of hyperparameters to sample from
param_distribs = {
    "model__n_hidden": [0, 1, 2, 3],                     # Number of hidden layers to try
    "model__n_neurons": np.arange(1, 100),               # Range of neurons per hidden layer (1 to 99)
    "model__learning_rate": reciprocal(3e-4, 3e-2)       # Log-uniform distribution between 0.0003 and 0.03
}

# Perform randomized hyperparameter search using 3-fold cross-validation
rnd_search_cv = RandomizedSearchCV(
    estimator=keras_reg,                # The KerasRegressor wrapper
    param_distributions=param_distribs,  # Distributions to sample from
    n_iter=10,                # Try 10 random combinations
    cv=3                      # Use 3-fold cross-validation
)

# Train the models on the training data using each sampled hyperparameter set
rnd_search_cv.fit(
    X_train, y_train,
    epochs=100,                                             # Train up to 100 epochs
    validation_data=(X_valid, y_valid),                     # Use validation data to monitor overfitting
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]  # Stop early if validation loss doesn't improve for 10 epochs
)


Epoch 1/100
242/242 [==============================] - 1s 3ms/step - loss: 7.3136 - mae: 2.1400 - val_loss: 6.6659 - val_mae: 2.0794
Epoch 2/100
242/242 [==============================] - 1s 2ms/step - loss: 6.6658 - mae: 2.0353 - val_loss: 6.0503 - val_mae: 1.9749
Epoch 3/100
242/242 [==============================] - 1s 2ms/step - loss: 6.0662 - mae: 1.9324 - val_loss: 5.4748 - val_mae: 1.8711
Epoch 4/100
242/242 [==============================] - 1s 2ms/step - loss: 5.4986 - mae: 1.8295 - val_loss: 4.9360 - val_mae: 1.7677
Epoch 5/100
242/242 [==============================] - 1s 2ms/step - loss: 4.9753 - mae: 1.7279 - val_loss: 4.4304 - val_mae: 1.6650
Epoch 6/100
242/242 [==============================] - 1s 3ms/step - loss: 4.4740 - mae: 1.6262 - val_loss: 3.9619 - val_mae: 1.5628
Epoch 7/100
242/242 [==============================] - 1s 2ms/step - loss: 4.0097 - mae: 1.5247 - val_loss: 3.5294 - val_mae: 1.4617
Epoch 8/100
242/242 [==============================] - 1s 2ms/step - 

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(model=<function build_model at 0x00000246817B9D00>, model__input_shape=(8,), model__learning_rate=0.01, model__n_hidden=2, model__n_neurons=30),
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000024685D77850>,
                                        'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [16]:
# Get the best combination of hyperparameters found during the randomized search
best_params = rnd_search_cv.best_params_

# Print the best parameters
print("Best hyperparameters found:", best_params)

# Get the best cross-validation score achieved during the randomized search
best_score = rnd_search_cv.best_score_

# Print the best score (negative mean squared error by default in Scikit-Learn for regressors)
print("Best cross-validation score:", best_score)

# Retrieve the actual best Keras model from the randomized search
model = rnd_search_cv.best_estimator_.model

Best hyperparameters found: {'model__learning_rate': 0.014102815358432825, 'model__n_hidden': 3, 'model__n_neurons': 35}
Best cross-validation score: 0.7773445754381015
